In [1]:
# Necessário possuir a biblioteca DuckDB, holidays e babel instaladas.
!pip install duckdb
!pip install babel
!pip install holidays
!pip install gdown

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/12.3 MB 11.8 MB/s eta 0:00:01
   ------------- -------------------------- 4.2/12.3 MB 12.3 MB/s eta 0:00:01
   ---------------------- ----------------- 6.8/12.3 MB 12.4 MB/s eta 0:00:01
   ----------------------------- ---------- 9.2/12.3 MB 12.3 MB/s eta 0:00:01
   ---------------------------------------  12.1/12.3 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 10.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 8.1 MB/s eta 0:00:00


In [2]:
import sys

# Getting Github files if this notebook is executed in a Google Colab environment.
if 'google.colab' in sys.modules:
  temp_folder = "etl_bank"

  !git clone -b 'dev' 'https://github.com/jpclarindo/etl_bank.git' $temp_folder
  !rsync -a $temp_folder/ .
  !rm -rf $temp_folder


Important! Google Drive Link for data source is required to execute this notebook. Please, put the link in gdrive_link.txt at the root.

In [1]:
import pandas as pd
import duckdb
import src.utils as utils

# Libraries to create date dimension
from datetime import datetime, timedelta
from babel.dates import format_date, format_datetime, format_time
import holidays

# Auxiliar libraries
import time
import json
import os, glob

In [2]:
# Init DuckDB connection.
con = duckdb.connect(database='database.duckdb', read_only=False)

# Creating cleaning function mapping for each attribute
cleaning_mapping = json.load(open('var/function_mapping.json','r',encoding='utf-8'))

# Creating date mapping
date_mapping = json.load(open('var/date_mapping.json','r',encoding='utf-8'))

# Creating relationships mapping
relationships = json.load(open('var/relationship_mapping.json','r',encoding='utf-8'))

Creating time and date dimension tables.

In [40]:
# @title
def create_time_dimension_table():
  time_list = []
  time_of_day = ['Early Morning', 'Morning', 'Afternoon', 'Night']
  time_of_day_pt = ['Madrugada', 'Manhã', 'Tarde', 'Noite']

  for i in range(86400):
    min_sec = i % 3600

    time_dict = {'tempo_id': i+1,
                'hora': i // 3600,
                'minuto': min_sec // 60,
                'segundo': min_sec % 60
                }

    time_dict['periodo_en'] = time_of_day[time_dict['hora'] // 6]
    time_dict['periodo_pt'] = time_of_day_pt[time_dict['hora'] // 6]
    time_dict['formatado'] = f"{time_dict['hora']:02d}:{time_dict['minuto']:02d}:{time_dict['segundo']:02d}"

    time_list.append(time_dict)


  time_df = pd.DataFrame(time_list)
  con.execute("""CREATE TABLE IF NOT EXISTS d_tempo
                 AS SELECT * FROM time_df""")

  print('Time dimension created and inserted in the database')
  return True

In [52]:
# @title
def create_date_dimension_table():
    date_list = []
    start_date =  datetime(1950, 1, 1)
    end_date = datetime(2050, 12, 31)
    delta = timedelta(days=1)
    count = 0

    while start_date <= end_date:
        count += 1
        
        date_dict = {'data_id': count,
                    'formatado': start_date.strftime('%Y-%m-%d'),
                    'ano': start_date.year,
                    'trimestre': format_date(start_date, format='Q' , locale='en_US'),
                    'nome_trimestre_en': format_date(start_date, format='QQQQ', locale='en_US'),
                    'nome_trimestre_pt': format_date(start_date, format='QQQQ', locale='pt_BR'),
                    'mes': start_date.month,
                    'nome_mes_en': format_date(start_date, format='MMMM', locale='en_US'),
                    'nome_mes_pt': format_date(start_date, format='MMMM', locale='pt_BR'),
                    'dia': start_date.day,
                    'nome_dia_en': format_date(start_date, format='EEEE', locale='en_US'),
                    'nome_dia_pt': format_date(start_date, format='EEEE', locale='pt_BR'),
                    'fim_de_semana': 1 if start_date.weekday() in [5, 6] else 0,
                    'feriado': 1 if start_date.strftime('%Y-%m-%d') in holidays.Brazil() else 0
        }
          
        start_date += delta
        date_list.append(date_dict)

    date_df = pd.DataFrame(date_list)
    con.execute("""CREATE TABLE IF NOT EXISTS d_data
                 AS SELECT * FROM date_df""")

    print('Date dimension created and inserted in the database')


ETL Processing - Extraction

In [20]:
def get_raw_data():
  #
  utils.download_data()

  # Adding files through a dict
  dfs_raw = {}

  file_list = glob.glob('data/*.csv')

  for csv_file in file_list:
    table_name = os.path.basename(csv_file).replace('.csv','')
    date_columns = date_mapping.get(table_name,[])

    df = pd.read_csv(csv_file, sep=',', encoding='utf-8')

    for col in df.columns:
        if col in date_columns:
            df[col] = pd.to_datetime(df[col], format='mixed')

            if df[col].dt.tz is not None:
                df[col] = df[col].dt.tz_localize(None)
            

    # Metadata for extraction
    df['_load_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    df['_file_name'] = os.path.basename(csv_file)

    dfs_raw[table_name] = df

  return dfs_raw

dfs_raw = get_raw_data()

Downloading...
From: https://drive.google.com/uc?id=16jF2hJHVOwikmgoEKs9wARnrE_j6Y7e2
To: D:\OneDrive\Documentos\GitHub\etl_bank\data.zip
100%|█████████████████████████████████████████████████████████████████████████████| 1.24M/1.24M [00:00<00:00, 3.79MB/s]


ETL - Cleaning

In [6]:
def cleaning_attributes(df):
    df_clean = df.copy()
    current_columns = set(df_clean.columns)

    for column, function_name in cleaning_mapping.items():
        if column in current_columns:
            if hasattr(utils, function_name):
                func = getattr(utils, function_name)
                print(f"   -> Expanding column '{column}' using '{function_name}'")

                # Aplica a função que retorna dict
                function_results = df_clean[column].apply(func)

                sample = function_results.dropna().iloc[0] if not function_results.dropna().empty else None

                if isinstance(sample, dict):
                # Transforma a coluna de dicts em um novo DataFrame de colunas
                    df_expanded = pd.json_normalize(function_results)
                    df_clean = pd.concat([df_clean, df_expanded], axis=1)
                    df_clean = df_clean.loc[:, ~df_clean.columns.duplicated(keep='last')]
                else:
                    df_clean[column] = function_results
                    
                #df_clean.drop(columns=[column], inplace=True)
            else:
                print(f"Error")       

    return df_clean

In [21]:
dfs_clean = {}

for table_name, df_raw in dfs_raw.items():
    print(f'Cleaning {table_name} attributes')
    dfs_clean[table_name] = cleaning_attributes(df_raw)

Cleaning agencias attributes
   -> Expanding column 'tipo_agencia' using 'clean_text'
   -> Expanding column 'cidade' using 'clean_text'
   -> Expanding column 'uf' using 'clean_text'
   -> Expanding column 'endereco' using 'get_address_dict'
Cleaning clientes attributes
   -> Expanding column 'primeiro_nome' using 'clean_text'
   -> Expanding column 'ultimo_nome' using 'clean_text'
   -> Expanding column 'tipo_cliente' using 'clean_text'
   -> Expanding column 'email' using 'normalize_email'
   -> Expanding column 'cep' using 'normalize_cep'
   -> Expanding column 'endereco' using 'get_address_dict'
   -> Expanding column 'data_nascimento' using 'get_age_dict'
Cleaning colaboradores attributes
   -> Expanding column 'primeiro_nome' using 'clean_text'
   -> Expanding column 'ultimo_nome' using 'clean_text'
   -> Expanding column 'email' using 'normalize_email'
   -> Expanding column 'cep' using 'normalize_cep'
   -> Expanding column 'endereco' using 'get_address_dict'
   -> Expanding c

ETL - Quality check

In [22]:
def repair_referential_integrity(dfs_dict_original, relationships_config):
    dfs_dict = dfs_dict_original.copy()
    counter = 0
    
    for rule in relationships_config:       
        source_col = rule['source_column']
        target_col = rule['target_column']
        
        df_source = dfs_dict.get(rule['source_table'])
        df_target = dfs_dict.get(rule['target_table'])
        
        # Identifying orphan keys
        source_keys = set(df_source[source_col].dropna().unique())
        target_keys = set(df_target[target_col].dropna().unique())
        orphan_keys = list(source_keys - target_keys)
        
        if not orphan_keys:
            print(f'The relationship between {rule['target_table']} and {rule['source_table']} complains integrity')
            counter += 1

            # If all relationships are consistent, return a boolean (for checking)
            if counter < len(relationships_config):
                continue
            else:
                return True
                
        print(f"Found inconsistencies ({len(orphan_keys)} rows) in '{rule['target_column']}' (Source table: {rule['source_table']})")

        # Creating a new dataframe with orphan keys
        df_orphans = pd.DataFrame({target_col: orphan_keys})
        
        for col in df_target.columns:
            if col == target_col:
                continue             

            # Get the type
            current_dtype = df_target[col].dtype
            
            # Fill the row with NI values
            if pd.api.types.is_numeric_dtype(current_dtype):
                df_orphans[col] = -1
            elif pd.api.types.is_datetime64_any_dtype(current_dtype):
                df_orphans[col] = pd.Timestamp('1900-01-01')
            else:
                df_orphans[col] = 'NI'

        dfs_dict[rule['target_table']] = pd.concat([df_target, df_orphans], ignore_index=True)

    return dfs_dict

In [23]:
dfs_clean_checked = repair_referential_integrity(dfs_clean, relationships)

The relationship between contas and transacoes complains integrity
Found inconsistencies (1 rows) in 'cod_cliente' (Source table: contas)
The relationship between agencias and contas complains integrity
The relationship between clientes and propostas_credito complains integrity
The relationship between colaboradores and propostas_credito complains integrity
The relationship between agencias and colaborador_agencia complains integrity


In [10]:
repair_referential_integrity(dfs_clean_checked, relationships)

The relationship between contas and transacoes complains integrity
The relationship between clientes and contas complains integrity
The relationship between agencias and contas complains integrity
The relationship between clientes and propostas_credito complains integrity
The relationship between colaboradores and propostas_credito complains integrity
The relationship between agencias and colaborador_agencia complains integrity


True

ETL - Loading

In [24]:
def load_to_staging_area(dfs_dict):
    """
    Persiste o dicionário de DataFrames como tabelas na camada Silver (Staging).
    """
    print(f"Connecting to base")
    
    # read_only=False permite escrita
    
    try:
        for table_name, df in dfs_dict.items():
            if df is None: continue
            
            db_name = f"stg_{table_name.lower()}"
            
            print(f"   -> Loading table: {db_name} ({len(df)} rows)")
            
            # Truque de Performance: Registrar View -> CTAS
            con.register('view_temp_pandas', df)
            
            con.execute(f"CREATE OR REPLACE TABLE {db_name} AS SELECT * FROM view_temp_pandas")
            
            con.unregister('view_temp_pandas')
            
        return True
        
    except Exception as e:
        print(f"Error")
        raise

In [25]:
load_to_staging_area(dfs_clean_checked)

Connecting to base
   -> Loading table: stg_agencias (10 rows)
   -> Loading table: stg_clientes (999 rows)
   -> Loading table: stg_colaboradores (100 rows)
   -> Loading table: stg_colaborador_agencia (100 rows)
   -> Loading table: stg_contas (999 rows)
   -> Loading table: stg_propostas_credito (2000 rows)
   -> Loading table: stg_transacoes (71999 rows)


True

In [29]:
def load_from_sql():    
    filenames = sorted([f for f in os.listdir('sql') if f.endswith('.sql')])

    for filename in filenames:
        full_path = os.path.join('sql', filename)
        print(f"   -> Running: {full_path} ...")        
        try:
            with open(full_path, 'r', encoding='utf-8') as f:
                query = f.read()                
            con.execute(query)
            
        except Exception as e:
            print(f"Error")
            raise e

In [51]:
load_from_sql()
create_time_dimension_table()
create_date_dimension_table()

   -> Running: sql\01_d_agencia.sql ...
   -> Running: sql\02_d_clientes.sql ...
   -> Running: sql\03_d_colaboradores.sql ...
   -> Running: sql\04_d_contas.sql ...
   -> Running: sql\05_f_transacoes.sql ...
   -> Running: sql\06_f_propostas.sql ...
Time dimension created and inserted in the database
Time dimension created and inserted in the database


OLAP

In [66]:
query_cidades = """
SELECT 
    d_cli.nome_cidade || ' - ' || d_cli.sigla_estado as cidade_uf,
    COUNT(f_t.transacao_id) AS "Qtd Transações",
    SUM(f_t.valor_transacao) AS "Volume Total",
    AVG(f_t.valor_transacao) AS "Ticket Médio"
FROM f_transacao f_t
JOIN d_conta d_con ON f_t.conta_id = d_con.conta_id
JOIN d_cliente d_cli ON d_con.cliente_id = d_cli.cliente_id
WHERE d_cli.sigla_estado <> 'NI'
GROUP BY d_cli.nome_cidade, d_cli.sigla_estado
ORDER BY "Volume Total" DESC
LIMIT 20; -- Pegando Top 20 para o gráfico não ficar poluição visual
"""

# Executa e traz para o Pandas
df_cidades = con.sql(query_cidades).df()

# Transforma colunas em linhas:
# Coluna 'Métrica' vai conter: "Qtd Transações", "Volume Total", "Ticket Médio"
# Coluna 'Valor' vai conter os números
df_long = df_cidades.melt(
    id_vars=['cidade_uf'], 
    value_vars=['Qtd Transações', 'Volume Total', 'Ticket Médio'],
    var_name='Métrica',
    value_name='Valor'
)

In [67]:
import altair as alt

# 1. Criar o Menu Dropdown
# As opções são os valores únicos da coluna 'Métrica' que criamos no melt
opcoes_metricas = ['Volume Total', 'Qtd Transações', 'Ticket Médio']
input_dropdown = alt.binding_select(options=opcoes_metricas, name="Selecione a Métrica: ")

# 2. Criar o Parâmetro de Seleção
selection = alt.selection_point(
    fields=['Métrica'], # O campo que será filtrado
    bind=input_dropdown, # Vincula ao menu visual
    value='Volume Total' # Valor inicial padrão
)

# 3. Construir o Gráfico
chart = alt.Chart(df_long).mark_bar().encode(
    # Eixo X: Cidades (ordenadas pelo valor atual dinamicamente)
    x=alt.X('cidade_uf', sort='-y', title='Cidade'),
    
    # Eixo Y: O valor numérico
    y=alt.Y('Valor', title='Valor da Métrica'),
    
    # Cor: Muda conforme a cidade para ficar bonito
    color=alt.Color('cidade_uf', legend=None),
    
    # Tooltip: Mostra os detalhes ao passar o mouse
    tooltip=[
        alt.Tooltip('cidade_uf', title='Cidade'),
        alt.Tooltip('Métrica', title='Indicador'),
        alt.Tooltip('Valor', format=',.2f') 
    ]
).add_params(
    selection # Adiciona o controle ao gráfico
).transform_filter(
    selection # A MÁGICA: Filtra os dados baseado no que você escolheu no menu
).properties(
    title='Análise de Performance por Cidade (Dinâmico)',
    width=700,
    height=400
).interactive()

# 4. Salvar ou Exibir
chart.display()('grafico_menu_dinamico.json')

alt.Chart(...)

TypeError: 'NoneType' object is not callable

In [53]:
con.execute('''
SELECT 
    d_dt.ano,
    d_dt.mes,
    COUNT(f_p.cod_proposta) AS qtd_propostas,
    SUM(f_p.valor_proposta) AS total_solicitado,
    SUM(f_p.valor_financiamento) AS total_aprovado
FROM f_proposta f_p
JOIN d_data d_dt ON f_p.data_id = d_dt.data_id
WHERE f_p.status = 'Aprovada' -- Filtro na Fato (ou dimensão degenerada)
GROUP BY d_dt.ano, d_dt.mes
ORDER BY d_dt.ano DESC, d_dt.mes DESC;
''')

In [55]:
con.execute("""
WITH metricas_colaborador AS (
    SELECT 
        d_ag.nome_agencia,
        d_col.nome_completo AS nome_colaborador,
        SUM(f_p.valor_financiamento) AS total_vendido
    FROM f_proposta f_p
    JOIN d_colaborador d_col ON f_p.colaborador_id = d_col.colaborador_id
    JOIN d_agencia d_ag ON d_col.agencia_id = d_ag.agencia_id
    WHERE f_p.status = 'Aprovada'
    GROUP BY d_ag.nome_agencia, d_col.nome_completo
),
ranking AS (
    SELECT 
        *,
        RANK() OVER (PARTITION BY nome_agencia ORDER BY total_vendido DESC) as rank_agencia
    FROM metricas_colaborador
)
SELECT * FROM ranking
WHERE rank_agencia <= 3
ORDER BY nome_agencia, rank_agencia;
""")

In [56]:
con.fetchall()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[('Agência Campinas', 'Marcela da Conceição', 1005076.37, 1),
 ('Agência Campinas', 'João Lucas Santos', 922978.65, 2),
 ('Agência Campinas', 'Pietra Caldeira', 861107.85, 3),
 ('Agência Digital', 'Pedro Lucas Rocha', 1092494.37, 1),
 ('Agência Digital', 'Alice Aragão', 870113.19, 2),
 ('Agência Digital', 'Pietro Souza', 870022.76, 3),
 ('Agência Florianópolis', 'Luigi Moraes', 1068916.12, 1),
 ('Agência Florianópolis', 'Lorena da Mata', 926005.4800000001, 2),
 ('Agência Florianópolis', 'Lara Santos', 772997.88, 3),
 ('Agência Jardins', 'Clarice Nascimento', 1015383.0900000001, 1),
 ('Agência Jardins', 'Elisa Correia', 804435.78, 2),
 ('Agência Jardins', 'Melissa Gonçalves', 712340.53, 3),
 ('Agência Matriz', 'Vitória Vieira', 1335624.8399999999, 1),
 ('Agência Matriz', 'Elisa Fogaça', 1165029.15, 2),
 ('Agência Matriz', 'Paulo Dias', 984494.41, 3),
 ('Agência Osasco', 'Vitor Hugo Dias', 1054735.42, 1),
 ('Agência Osasco', 'Lucas Gabriel da Costa', 1001324.5000000001, 2),
 ('Agência Os

In [57]:
import altair as alt

In [59]:
# 1. Executa a Query OLAP e traz para o Pandas
# (Assumindo que sua conexão 'con' ou 'con_dw' está aberta)
query_trend = """
SELECT 
    d_dt.ano,
    d_dt.mes,
    -- Criamos uma coluna de data real para facilitar o eixo X do gráfico
    make_date(d_dt.ano, d_dt.mes, 1) as data_referencia,
    COUNT(f_p.cod_proposta) AS qtd_propostas,
    SUM(f_p.valor_proposta) AS total_solicitado,
    SUM(f_p.valor_financiamento) AS total_aprovado
FROM f_proposta f_p
JOIN d_data d_dt ON f_p.data_id = d_dt.data_id
WHERE f_p.status = 'Aprovada'
GROUP BY d_dt.ano, d_dt.mes
ORDER BY d_dt.ano, d_dt.mes;
"""

df_trend = con.sql(query_trend).df()

# 2. Construção do Gráfico com Altair

# Base comum para os dois gráficos
base = alt.Chart(df_trend).encode(
    x=alt.X('yearmonth(data_referencia):T', title='Data (Mês/Ano)')
)

# Camada 1: Barras (Volume Financeiro Aprovado)
barras = base.mark_bar(opacity=0.7, color='#4c78a8').encode(
    y=alt.Y('total_aprovado:Q', title='Volume Aprovado (R$)', axis=alt.Axis(format='$,.2f')),
    tooltip=[
        alt.Tooltip('data_referencia', title='Data', format='%m/%Y'),
        alt.Tooltip('total_aprovado', title='Total Aprovado', format='$,.2f'),
        alt.Tooltip('qtd_propostas', title='Qtd Propostas')
    ]
)

# Camada 2: Linha (Quantidade de Propostas)
# Usamos 'resolve_scale' depois para criar o eixo duplo se necessário, 
# mas aqui vou colocar como uma linha vermelha de destaque.
linha = base.mark_line(point=True, color='#e45756').encode(
    y=alt.Y('qtd_propostas:Q', title='Quantidade de Propostas'),
    tooltip=[
        alt.Tooltip('data_referencia', title='Data', format='%m/%Y'),
        alt.Tooltip('qtd_propostas', title='Qtd. Propostas')
    ]
)

# 3. Combinação (Layering) com eixos independentes
grafico_final = alt.layer(barras, linha).resolve_scale(
    y='independent' # Cria dois eixos Y (um na esquerda, um na direita)
).properties(
    title='Evolução Mensal: Volume Aprovado vs. Quantidade de Propostas',
    width=700,
    height=400
).interactive()

# 4. Exibir ou Salvar
#rafico_final.save('evolucao_propostas.json')
grafico_final.display() # Se estiver no Jupyter

alt.LayerChart(...)